# 4. Holdout creation

### Import the needed packages

In [13]:
import os
from ydata.labs.datasources import DataSources

from ydata.dataset import Dataset
from ydata.metadata import Metadata
from ydata.dataset.holdout import Holdout

from functions.saving_functions import read_file, save_file

#### Get environment variables values

In [14]:
pipeline_parameters = read_file(file_path='pipeline_parameters.pkl')

### Read the dataset & Metadata

In [15]:
metadata = Metadata.load('proc_metadata.pkl')
#dataset = read_file('dataset.pkl')

dataset = DataSources.get(uid=pipeline_parameters['preprocessed_id']).read()

In [16]:
dataset.head()

,Gender,Senior Citizen,Partner,Dependents,Tenure,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,...,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn,CLTV,Avg Montly charges per city,Avg Total charges per city
0,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,...,No,Month-to-month,Yes,Mailed check,53.85,108.15,1,3239,62.42377,2130.760691
1,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,...,No,Month-to-month,Yes,Electronic check,70.70,151.65,1,2701,62.42377,2130.760691
2,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,...,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,1,5372,62.42377,2130.760691
3,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,...,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,1,5003,62.42377,2130.760691
4,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,...,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.30,1,5340,62.42377,2130.760691


## Get an holdout sample

In [17]:
#Creating the holdout set
#Reading the holdout size from the pipeline definition
holdout_ratio = float(os.environ.get('HOLDOUT_RATIO', 0.15))

In [18]:
data = dataset.to_pandas()
holdout_set = data.sample(frac=holdout_ratio)
train_set = data.drop(holdout_set.index)

In [ ]:
train_set = Dataset(train_set)
holdout_set = Dataset(holdout_set)

In [19]:
#recalculate the metadata
metadata_train = Metadata(train_set)
metadata_holdout = Metadata(holdout_set)

[                                        ] | 0% Completed | 265.50 us

/opt/conda/lib/python3.8/site-packages/ydata/metadata/metadata.py:143: UserWarning: Datasets other than Timeseries don't make use of dataset_attrs
  warn("Datasets other than Timeseries don't make use of dataset_attrs")


[########################################] | 100% Completed | 102.07 ms
[########################################] | 100% Completed | 830.49 ms
[                                        ] | 0% Completed | 181.70 us

/opt/conda/lib/python3.8/site-packages/ydata/metadata/metadata.py:143: UserWarning: Datasets other than Timeseries don't make use of dataset_attrs
  warn("Datasets other than Timeseries don't make use of dataset_attrs")


[########################################] | 100% Completed | 101.91 ms
[########################################] | 100% Completed | 725.61 ms


In [20]:
metadata_holdout = Metadata(holdout_set)

[                                        ] | 0% Completed | 166.40 us

/opt/conda/lib/python3.8/site-packages/ydata/metadata/metadata.py:143: UserWarning: Datasets other than Timeseries don't make use of dataset_attrs
  warn("Datasets other than Timeseries don't make use of dataset_attrs")


[########################################] | 100% Completed | 101.52 ms
[########################################] | 100% Completed | 725.66 ms


In [21]:
metadata_holdout

In [22]:
print(metadata_holdout)

Metadata Summary 
 
Dataset type: TABULAR
Dataset attributes: 
Number of columns: 23
Duplicate rows: 1
Target column: 

Column detail: 
                         Column    Data type Variable type
0                        Gender  categorical        string
1                Senior Citizen  categorical        string
2                       Partner  categorical        string
3                    Dependents  categorical        string
4                        Tenure    numerical           int
5                 Phone Service  categorical        string
6                Multiple Lines  categorical        string
7              Internet Service  categorical        string
8               Online Security  categorical        string
9                 Online Backup  categorical        string
10            Device Protection  categorical        string
11                 Tech Support  categorical        string
12                 Streaming TV  categorical        string
13             Streaming Movies  categ

## Create pipeline outputs

In [23]:
pipeline_parameters= read_file('pipeline_parameters.pkl')
pipeline_parameters['holdout'] = holdout_ratio
save_file(pipeline_parameters, 'pipeline_parameters.pkl')

In [24]:
metadata_train.save('metadata_train.pkl')
metadata_holdout.save('metadata_holdout.pkl')

save_file(train_set, 'train_set.pkl')
save_file(holdout_set, 'holdout_set.pkl')